# 변이형 오토인코더 훈련 - 얼굴 데이터셋

## 라이브러리 임포트

In [1]:
import os
from glob import glob
import numpy as np

from models.VAE import VariationalAutoencoder
from keras.preprocessing.image import ImageDataGenerator

Using TensorFlow backend.


In [2]:
# run params
section = 'vae'
run_id = '0001'
data_name = 'faces'
RUN_FOLDER = 'run/{}/'.format(section)
RUN_FOLDER += '_'.join([run_id, data_name])

if not os.path.exists(RUN_FOLDER):
    os.mkdir(RUN_FOLDER)
    os.mkdir(os.path.join(RUN_FOLDER, 'viz'))
    os.mkdir(os.path.join(RUN_FOLDER, 'images'))
    os.mkdir(os.path.join(RUN_FOLDER, 'weights'))

mode =  'build' #'load' #


DATA_FOLDER = './data/celeb/'

## 데이터 적재

In [3]:
INPUT_DIM = (128,128,3)
BATCH_SIZE = 32

filenames = np.array(glob(os.path.join(DATA_FOLDER, '*/*.jpg')))

NUM_IMAGES = len(filenames)

In [4]:
data_gen = ImageDataGenerator(rescale=1./255)

data_flow = data_gen.flow_from_directory(DATA_FOLDER
                                         , target_size = INPUT_DIM[:2]
                                         , batch_size = BATCH_SIZE
                                         , shuffle = True
                                         , class_mode = 'input'
                                         , subset = "training"
                                            )

Found 202599 images belonging to 1 classes.


## 모델 만들기

In [5]:
vae = VariationalAutoencoder(
                input_dim = INPUT_DIM
                , encoder_conv_filters=[32,64,64, 64]
                , encoder_conv_kernel_size=[3,3,3,3]
                , encoder_conv_strides=[2,2,2,2]
                , decoder_conv_t_filters=[64,64,32,3]
                , decoder_conv_t_kernel_size=[3,3,3,3]
                , decoder_conv_t_strides=[2,2,2,2]
                , z_dim=200
                , use_batch_norm=True
                , use_dropout=True)

if mode == 'build':
    vae.save(RUN_FOLDER)
else:
    vae.load_weights(os.path.join(RUN_FOLDER, 'weights/weights.h5'))

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [6]:
vae.encoder.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
encoder_input (InputLayer)      (None, 128, 128, 3)  0                                            
__________________________________________________________________________________________________
encoder_conv_0 (Conv2D)         (None, 64, 64, 32)   896         encoder_input[0][0]              
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, 64, 64, 32)   128         encoder_conv_0[0][0]             
__________________________________________________________________________________________________
leaky_re_lu_1 (LeakyReLU)       (None, 64, 64, 32)   0           batch_normalization_1[0][0]      
__________________________________________________________________________________________________
dropout_1 

In [7]:
vae.decoder.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
decoder_input (InputLayer)   (None, 200)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 4096)              823296    
_________________________________________________________________
reshape_1 (Reshape)          (None, 8, 8, 64)          0         
_________________________________________________________________
decoder_conv_t_0 (Conv2DTran (None, 16, 16, 64)        36928     
_________________________________________________________________
batch_normalization_5 (Batch (None, 16, 16, 64)        256       
_________________________________________________________________
leaky_re_lu_5 (LeakyReLU)    (None, 16, 16, 64)        0         
_________________________________________________________________
dropout_5 (Dropout)          (None, 16, 16, 64)        0         
__________

## 모델 훈련

In [8]:
LEARNING_RATE = 0.0005
R_LOSS_FACTOR = 10000
EPOCHS = 200
PRINT_EVERY_N_BATCHES = 100
INITIAL_EPOCH = 0

In [9]:
vae.compile(LEARNING_RATE, R_LOSS_FACTOR)

In [10]:
vae.train_with_generator(     
    data_flow
    , epochs = EPOCHS
    , steps_per_epoch = NUM_IMAGES / BATCH_SIZE
    , run_folder = RUN_FOLDER
    , print_every_n_batches = PRINT_EVERY_N_BATCHES
    , initial_epoch = INITIAL_EPOCH
)

Instructions for updating:
Use tf.cast instead.
Epoch 1/200
   8/6331 [..............................] - ETA: 2:09:27 - loss: 1259.6693 - vae_r_loss: 1069.6871 - vae_kl_loss: 189.9822

/home/jupyter/GDL_code/env/lib/python3.5/site-packages/keras/callbacks.py:122: UserWarning: Method on_batch_end() is slow compared to the batch update (0.141168). Check your callbacks.
  % delta_t_median)


6332/6331 [==============================] - 5076s 802ms/step - loss: 275.9733 - vae_r_loss: 213.6287 - vae_kl_loss: 62.3446

Epoch 00001: saving model to run/vae/0001_faces/weights/weights-001-275.97.h5

Epoch 00001: saving model to run/vae/0001_faces/weights/weights.h5
Epoch 2/200
   2/6331 [..............................] - ETA: 19:24 - loss: 245.8827 - vae_r_loss: 187.6025 - vae_kl_loss: 58.2802

/home/jupyter/GDL_code/env/lib/python3.5/site-packages/keras/callbacks.py:122: UserWarning: Method on_batch_end() is slow compared to the batch update (0.153775). Check your callbacks.
  % delta_t_median)


6332/6331 [==============================] - 1713s 271ms/step - loss: 233.9473 - vae_r_loss: 174.7609 - vae_kl_loss: 59.1864

Epoch 00002: saving model to run/vae/0001_faces/weights/weights-002-233.95.h5

Epoch 00002: saving model to run/vae/0001_faces/weights/weights.h5
Epoch 3/200
   2/6331 [..............................] - ETA: 20:23 - loss: 238.1785 - vae_r_loss: 176.7546 - vae_kl_loss: 61.4238

...

6332/6331 [==============================] - 1757s 277ms/step - loss: 216.9430 - vae_r_loss: 157.9114 - vae_kl_loss: 59.0317

Epoch 00198: saving model to run/vae/0001_faces/weights/weights-198-216.94.h5

Epoch 00198: saving model to run/vae/0001_faces/weights/weights.h5
Epoch 199/200
   2/6331 [..............................] - ETA: 19:01 - loss: 211.4890 - vae_r_loss: 152.3988 - vae_kl_loss: 59.0902

/home/jupyter/GDL_code/env/lib/python3.5/site-packages/keras/callbacks.py:122: UserWarning: Method on_batch_end() is slow compared to the batch update (0.163861). Check your callbacks.
  % delta_t_median)


6332/6331 [==============================] - 1760s 278ms/step - loss: 216.8629 - vae_r_loss: 157.8460 - vae_kl_loss: 59.0170

Epoch 00199: saving model to run/vae/0001_faces/weights/weights-199-216.86.h5

Epoch 00199: saving model to run/vae/0001_faces/weights/weights.h5
Epoch 200/200
   2/6331 [..............................] - ETA: 19:34 - loss: 223.4222 - vae_r_loss: 164.4930 - vae_kl_loss: 58.9292

/home/jupyter/GDL_code/env/lib/python3.5/site-packages/keras/callbacks.py:122: UserWarning: Method on_batch_end() is slow compared to the batch update (0.143169). Check your callbacks.
  % delta_t_median)


6332/6331 [==============================] - 1757s 277ms/step - loss: 216.9563 - vae_r_loss: 157.9158 - vae_kl_loss: 59.0405

Epoch 00200: saving model to run/vae/0001_faces/weights/weights-200-216.96.h5

Epoch 00200: saving model to run/vae/0001_faces/weights/weights.h5
